## Алгоритмы в биоинформатике
### ДЗ 2.2. Глобальное выравнивание с афинными штрафами
#### Буклей Григорий

Задание: Придумайте две последовательности и 2 разных набора штрафов за открытие и продолжение гэпа, чтобы получились разные выравнивания.

$\textbf{Описание функции:}$

$\text{affine_gap_alignment}\textit{(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):}$

$\textit{seq1, seq2}$ - последовательности для выравнивания

$\textit{weight_match}$ - вес совпадения

$\textit{weight_mismatch}$ - вес несовпадения

$\textit{open_gap_penalty}$ - штраф за открытие гэпа

$\textit{continue_gap_penalty}$ - штраф за продолжение гэпа

Возьмём следующие строки


$$seq_1 = \text{ABXXMYYYBB}, \;\; seq_2 = \text{ABMBB}$$

Пусть в первом случе weight_match = 1, weight_mismatch = 1, open_gap_penalty = 1, continue_gap_penalty = 1

Тогда глобальным выравниванием для них является:
$$\text{A B X X M Y Y Y B B}$$ $$\text{A B _ _} \;\;\; \text{M _ _} \;\; \text{_ B B}$$

Но если weight_match = 1, weight_mismatch = 1, open_gap_penalty = 10, continue_gap_penalty = 1

Тогда глобальным выравниванием будет:
$$\text{A B X X M Y Y Y B B}$$ $$\text{A B _ _} \;\;\; \text{_ _ _} \;\; \text{M B B}$$


In [1]:
def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
    topTable = [] # creates/extends gaps in the 1st sequence
    middleTable = [] # extends matches and mismatches
    lowTable = [] # creates/extends gaps in the 2nd sequence
    prev = [] # array for fathers
    max_int = 10** 6
    for i in range(len(seq1) + 1): # filling array with zeroes
        lowTable.append([0] * (len(seq2) + 1))
        middleTable.append([0] * (len(seq2) + 1))
        topTable.append([0] * (len(seq2) + 1))
        prev.append([(-1, -1)] * (len(seq2) + 1))

    # middle init
    for i in range(1, len(seq1) + 1):
        middleTable[i][0] = -open_gap_penalty - (i - 1) * continue_gap_penalty
        prev[i][0] = (i - 1, 0)
    for j in range(1, len(seq2) + 1):
        middleTable[0][j] = -open_gap_penalty - (j - 1) * continue_gap_penalty
        prev[0][j] = (0, j - 1)

    # top init
    for i in range(1, len(seq1) + 1):
        topTable[i][0] = -open_gap_penalty * len(seq1) * max_int
    for j in range(1, len(seq2) + 1):
        topTable[0][j] = -open_gap_penalty - (j - 1) * continue_gap_penalty

    # low init
    for i in range(1, len(seq1) + 1):
        lowTable[i][0] = -open_gap_penalty - (i - 1) * continue_gap_penalty
    for j in range(1, len(seq2) + 1):
        lowTable[0][j] = -open_gap_penalty * len(seq1) * max_int

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            topTable[i][j] = max(topTable[i][j - 1] - continue_gap_penalty,
                                 middleTable[i][j - 1] - open_gap_penalty)
            lowTable[i][j] = max(lowTable[i - 1][j] - continue_gap_penalty,
                                 middleTable[i - 1][j] - open_gap_penalty)
            if seq1[i - 1] == seq2[j - 1]:
                eq = weight_match
            else:
                eq = -weight_mismatch

            middleTable[i][j] = max(middleTable[i - 1][j - 1] + eq,
                                    lowTable[i][j],
                                    topTable[i][j])

            if middleTable[i][j] == middleTable[i - 1][j - 1] + eq:
                prev[i][j] = (i - 1, j - 1)
            elif middleTable[i][j] == lowTable[i][j]:
                prev[i][j] = (i - 1, j)
            else:
                prev[i][j] = (i, j - 1)

    ans1, ans2 = "", ""

    curPosition = (len(seq1), len(seq2))
    while curPosition != (0, 0):
        if prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1] - 1):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += seq2[curPosition[1] - 1]
        elif prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1]):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += "-"
        else:
            ans1 += "-"
            ans2 += seq2[curPosition[1] - 1]

        curPosition = prev[curPosition[0]][curPosition[1]]

    return ans1[::-1], ans2[::-1]

In [2]:
seq1 = "ABXXMYYYBB"
seq2 = "ABMBB"
ans1, ans2 = affine_gap_alignment(seq1, seq2, 1, 1, 1, 1)
print("First case: ")
print(ans1)
print(ans2)

ans3, ans4 = affine_gap_alignment(seq1, seq2, 1, 1, 10, 1)
print("Second case: ")
print(ans3)
print(ans4)

First case: 
ABXXMYYYBB
AB--M---BB
Second case: 
ABXXMYYYBB
AB-----MBB
